In [113]:
import os
import time

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from gams import GamsWorkspace

# import seaborn as sns
import argparse

import sys

In [114]:
import os
from gams import *

# Assuming your Jupyter notebook and the GDX file are in the same directory 'crab'
# and your Jupyter server was started in the directory containing 'crab'

# Get the current working directory of the Jupyter Notebook
notebook_directory = %pwd

# The path to your GDX file relative to the notebook
gdx_file_path = os.path.join(notebook_directory, "SimulationResults_CGE.gdx")

print("Attempting to load GDX file from:", gdx_file_path)

# Create GAMS Workspace
WS = GamsWorkspace()

try:
    # Attempt to load the GDX file using the full path
    db = WS.add_database_from_gdx(gdx_file_path)
    print("Database loaded successfully.")
except Exception as e:
    print("Error loading GDX file:", e)




Attempting to load GDX file from: /Users/ataberna/Library/CloudStorage/OneDrive-DelftUniversityofTechnology/Documents/GitHub/CRAB_EU_EMS_coupling/crab/SimulationResults_CGE.gdx
Database loaded successfully.


In [115]:


all_data = []
sectors = ['Agriculture', 'Industry_capital', 'Industry_rest', 'Construction',
           'Utilities', 'Logistics', 'Transport', 'Private_Services', 'Public_Services']
current_year = str(2020)
regions = ['NL11']

# Assuming 'db' is a list of symbol objects with a 'name' attribute and iterates to give records
for i, symbol in enumerate(db):
    filtered_data = []
    print(symbol.name)
    for record in symbol:
        region, sector, year = record.keys
        if region in regions and sector in sectors and year == current_year:
            print(region, sector, year, record.value)
            # Create a dictionary for each record and append to filtered_data
            filtered_data.append({'region': region, 'sector': sector, f'{symbol.name}': record.value})

    # If filtered_data is not empty, create a DataFrame and append it to all_data
    if filtered_data:
        all_data.append(pd.DataFrame(filtered_data))

# Combine all DataFrames into one DataFrame using reduce and merge
from functools import reduce

# Use functools.reduce to perform a cumulative merge operation
# Use outer join to ensure all data is kept and aligned without repetition
combined_data = reduce(lambda left, right: pd.merge(left, right, on=['region', 'sector'], how='outer'), all_data)

# Now combined_data should have all the data aligned without NaNs if the 'region' and 'sector' pairs match up
combined_data


K_XD_rate


NL11 Agriculture 2020 1.8703413761967782
NL11 Industry_capital 2020 0.6893987463816398
NL11 Industry_rest 2020 2.910588991160614
NL11 Construction 2020 1.274565540347857
NL11 Utilities 2020 2.5160062137103125
NL11 Logistics 2020 2.0479098643101348
NL11 Transport 2020 0.8833702698597707
NL11 Private_Services 2020 2.267256348929262
NL11 Public_Services 2020 1.3275594027012954
Consumption
NL11 Agriculture 2020 385.52163969172216
NL11 Industry_capital 2020 954.529595619173
NL11 Industry_rest 2020 4190.907766655704
NL11 Construction 2020 706.4378753569054
NL11 Utilities 2020 474.2155427692498
NL11 Logistics 2020 1666.3059307098972
NL11 Transport 2020 447.00476280173825
NL11 Private_Services 2020 3461.8944309933186
NL11 Public_Services 2020 3296.173207189882


,region,sector,K_XD_rate,Consumption
0,NL11,Agriculture,1.870341,385.521640
1,NL11,Industry_capital,0.689399,954.529596
2,NL11,Industry_rest,2.910589,4190.907767
3,NL11,Construction,1.274566,706.437875
4,NL11,Utilities,2.516006,474.215543
5,NL11,Logistics,2.047910,1666.305931
6,NL11,Transport,0.883370,447.004763
7,NL11,Private_Services,2.267256,3461.894431
8,NL11,Public_Services,1.327559,3296.173207


In [116]:
# in combined_data, if sector == 'Industry_capital', set 'Consumption' to 0
combined_data.loc[combined_data['sector'] == 'Industry_capital', 'Consumption'] = 0
# in combined_data. normalize the columns Consumption so that it sums to 1
combined_data['Consumption'] = combined_data['Consumption'] / combined_data['Consumption'].sum()

In [117]:
combined_data
# drop the columns 'region' 
combined_data = combined_data.drop(columns=['region'])
# save as cge_input.csv
combined_data.to_csv('cge_input.csv', index=False)

In [118]:
# read combined_data 
cge_input = pd.read_csv('cge_input.csv')

In [125]:
KL_RATIO  =  pd.read_csv("cge_input.csv")
KL_RATIO.set_index("sector", inplace=True)
KL_RATIO = KL_RATIO.to_dict()["K_XD_rate"]

### Now let's work on the output

In [120]:
import pickle

In [121]:
def load_model(filename):
    with open(filename, 'rb') as f:
        model = pickle.load(f)
    return model

In [122]:
# load model  crab/saved_model_0.pkl
model = load_model('saved_model_0.pkl')


In [123]:
# get 
macro_variables = model.datacollector.get_model_vars_dataframe()
micro_variables = model.datacollector.get_agent_vars_dataframe()

In [138]:
micro_variables

,Step,AgentID,Type,Capital amount
0,1,1,Industry_capital,80.0
1,1,2,Industry_capital,76.0
2,1,3,Industry_capital,76.0
3,1,4,Industry_capital,72.0
4,1,5,Industry_capital,76.0
...,...,...,...,...
226015,20,11520,Transport,4.0
226016,20,11521,Utilities,9.0
226017,20,11522,Private_Services,11.0
226018,20,11523,Public_Services,7.0


In [128]:
sectors = ['Agriculture', 'Industry_capital', 'Industry_rest', 'Construction',
           'Utilities', 'Logistics', 'Transport', 'Private_Services', 'Public_Services']

pattern = '(' + '|'.join(sectors) + ')'  # create a pattern that matches any of the sectors

# use the str.extract method to extract the matching part of the string
micro_variables['Type'] = micro_variables['Type'].str.extract(pattern, expand=False)

In [137]:
macro_variables['Step'] = macro_variables.index + 1
micro_variables.reset_index(inplace=True)

In [155]:
micro_variables_grouped

,Step,Type,Capital amount
0,1,Agriculture,19.420000
1,1,Construction,19.500000
2,1,Industry_capital,77.626667
3,1,Industry_rest,29.213333
4,1,Logistics,19.500000
...,...,...,...
175,20,Logistics,24.120000
176,20,Private_Services,35.350000
177,20,Public_Services,29.430000
178,20,Transport,8.053333


In [143]:
t = 2020
# get the Step where the year is t
first_step = macro_variables[macro_variables['Year'] == t]['Step'][0]
last_step = macro_variables[macro_variables['Year'] == t]['Step'].iloc[-1]

In [189]:
# The path to your GDX file relative to the notebook
gdx_file_path_out = os.path.join(notebook_directory, "SimulationResults_ABM.gdx")

print("Attempting to load GDX file from:", gdx_file_path_out)

# Create GAMS Workspace

try:
    # Attempt to load the GDX file using the full path
    db_out = WS.add_database_from_gdx(gdx_file_path_out)
    print("Database loaded successfully.")
except Exception as e:
    print("Error loading GDX file:", e)

Attempting to load GDX file from: /Users/ataberna/Library/CloudStorage/OneDrive-DelftUniversityofTechnology/Documents/GitHub/CRAB_EU_EMS_coupling/crab/SimulationResults_ABM.gdx
Database loaded successfully.


In [191]:
for symbol in db_out:
    print(symbol.name)
    for record in symbol:
                     
        print(record.keys, record.value)

K_XD_rate
['NL11', 'Agriculture', '2020'] 1.8703413761967782
['NL11', 'Agriculture', '2021'] 1.8703484005540059
['NL11', 'Agriculture', '2022'] 1.8703553919707616
['NL11', 'Agriculture', '2023'] 1.87036235064999
['NL11', 'Agriculture', '2024'] 1.8703692768117592
['NL11', 'Agriculture', '2025'] 1.870376170675208
['NL11', 'Agriculture', '2026'] 1.8703830324585469
['NL11', 'Agriculture', '2027'] 1.8703898623790902
['NL11', 'Agriculture', '2028'] 1.870396660653253
['NL11', 'Agriculture', '2029'] 1.8704034274965384
['NL11', 'Agriculture', '2030'] 1.8704101631235823
['NL11', 'Agriculture', '2031'] 1.8704168677481137
['NL11', 'Agriculture', '2032'] 1.8704235415829897
['NL11', 'Agriculture', '2033'] 1.8704301848401945
['NL11', 'Agriculture', '2034'] 1.8704367977308487
['NL11', 'Agriculture', '2035'] 1.8704433804651892
['NL11', 'Agriculture', '2036'] 1.8704499332526248
['NL11', 'Agriculture', '2037'] 1.8704564563016826
['NL11', 'Agriculture', '2038'] 1.870462949820064
['NL11', 'Agriculture', '2

In [176]:
columns_out = ['Capital amount']
columns_names = ['K_firm_loss']
data_out = {}
n = 0
# groupby by micro_variables
micro_variables_grouped = micro_variables.groupby(['Step', 'Type'])[columns_out].mean().reset_index()
# for each column in micro_variables, get the % change between first step and last step , for each sector in the column 'Type'
for column in micro_variables_grouped.columns:
    if column in columns_out:
        data_out[columns_names[n]] = {}
        # get the % change between first step and last step, for each sector in the column 'Type'
        for sector in sectors:
            df_sector = micro_variables_grouped[(micro_variables_grouped['Type'] == sector)]
            first_value = df_sector[df_sector['Step'] == first_step][column].values[0]
            last_value = df_sector[df_sector['Step'] == last_step][column].values[0]
            # perc change
            perc_change = (last_value - first_value) / first_value * 100
            data_out[columns_names[n]][sector] = perc_change
        n += 1

            


In [188]:
for symbol in db_out:
    print(symbol.name)
    if symbol.name in data_out:
        data_to_add = data_out[symbol.name]
        for record in symbol:
            if record.keys[0] == 'NL11':
                if record.keys[2] == str(t):
                    # record_keys[1] is the sector, that is the same name in data_out
                    # I need to match them to update the value
                    sector = record.keys[1]
                    record.value = data_to_add[sector]
                    
                
                    print(record.keys, record.value)


        
    

I_additional_time_in
K_firm_loss
['NL11', 'Agriculture', '2020'] -49.691040164778585
['NL11', 'Industry_capital', '2020'] -47.40638955685331
['NL11', 'Industry_rest', '2020'] -4.176175262437251
['NL11', 'Construction', '2020'] -49.69230769230769
['NL11', 'Utilities', '2020'] -42.60958205912335
['NL11', 'Logistics', '2020'] 23.692307692307697
['NL11', 'Transport', '2020'] -58.4308327598073
['NL11', 'Private_Services', '2020'] 80.54136874361596
['NL11', 'Public_Services', '2020'] 51.07802874743326


In [150]:

gdx_file_path_out = os.path.join(script_directory, output_filnename)
# save a sa gdx
db.export(gdx_file_path_out)




Attempting to load GDX file from: /Users/ataberna/Library/CloudStorage/OneDrive-DelftUniversityofTechnology/Documents/GitHub/CRAB_EU_EMS_coupling/crab/SimulationResults_ABM.gdx
Database loaded successfully.


In [147]:
per_changes

{'Capital amount': -0.2241845621247363}

In [ ]:
# need to load the GDX 

In [134]:
macro_variables

,Flood,Year,HH consumption,Regional demand,Export demand,Unemployment rate,Min wage,Avg wage,Step
0,False,2020,6519.444235,6519.445,0.0,0.87,0.600,0.999572,1
1,False,2020,7362.529235,7362.529,0.0,0.69,0.600,1.037128,2
2,False,2020,8271.953235,8271.954,0.0,0.51,0.624,1.038769,3
3,False,2020,8930.557591,8930.558,0.0,0.35,0.624,1.038698,4
4,False,2020,8882.712377,9226.212,0.0,0.31,0.624,1.033876,5
5,False,2020,9464.487695,9577.488,0.0,0.19,0.618,1.033629,6
6,False,2020,9606.575075,9661.575,0.0,0.17,0.618,1.033472,7
7,False,2020,9598.750702,9656.751,0.0,0.17,0.618,1.033644,8
8,False,2020,9344.514123,9682.512,0.0,0.19,0.618,1.034409,9
9,False,2020,9497.947489,9633.948,0.0,0.19,0.618,1.034554,10


In [32]:
df = pd.DataFrame(filtered_data)

In [31]:
df

,region,sector,K_XD_rate,Consumption
0,NL11,Agriculture,1.870341,NaN
1,NL11,Industry_capital,0.689399,NaN
2,NL11,Industry_rest,2.910589,NaN
3,NL11,Construction,1.274566,NaN
4,NL11,Utilities,2.516006,NaN
5,NL11,Logistics,2.047910,NaN
6,NL11,Transport,0.883370,NaN
7,NL11,Private_Services,2.267256,NaN
8,NL11,Public_Services,1.327559,NaN
9,NL11,Agriculture,NaN,385.521640


In [15]:
interested_symbols = ['K_index', 'L_index']  # Add more symbols as needed
interested_years = [str(2015)]  # Add more years as needed
interested_regions = ['Netherlands']
# Assuming 'db' is your GAMS database object
filtered_data = []

# Iterate through all symbols in the database
for symbol in db:
    if symbol.name in interested_symbols:
        print(f"Symbol: {symbol.name}")
        for rec in symbol:
            if len(rec.keys) == 4:  # Assuming the order is region, sector, education, year
                region, sector, edu, year = rec.keys
            else:  # If 'education' is not present, assign NaN to edu
                region, sector, year = rec.keys
                edu = np.nan  # Use numpy's NaN for missing values
            
            # Now include the 'education' dimension in your filtering and data appending
            if region in interested_regions and year in interested_years:
                filtered_data.append({
                    'Region': region,
                    'Year': year,
                    'Sector': sector,
                    'Education': edu,  # Include 'education' regardless of its presence
                    'Value': rec.value
                })

# Convert the filtered data to a pandas DataFrame
df = pd.DataFrame(filtered_data)


Symbol: K_index
Symbol: L_index


In [16]:
df

,Region,Year,Sector,Education,Value
0,Netherlands,2015,CPA_A01,NaN,2.220446e-14
1,Netherlands,2015,CPA_A02,NaN,2.220446e-14
2,Netherlands,2015,CPA_C10-C12,NaN,2.220446e-14
3,Netherlands,2015,CPA_C13-C15,NaN,-1.110223e-14
4,Netherlands,2015,CPA_C17,NaN,2.220446e-14
5,Netherlands,2015,CPA_C18,NaN,-1.110223e-14
6,Netherlands,2015,CPA_C21,NaN,-2.220446e-14
7,Netherlands,2015,CPA_C26,NaN,2.220446e-14
8,Netherlands,2015,CPA_C30,NaN,2.220446e-14
9,Netherlands,2015,CPA_C33,NaN,-1.110223e-14


In [ ]:



# -- READ FILES -- #
HH_attributes = pd.read_csv("Input/HH_attributes.csv", index_col=0)
firm_flood_depths = pd.read_csv("Input/Firm_attributes.csv", index_col=0)
PMT_weights = pd.read_csv("Input/PMT_weights.csv", index_col=0)

# Extract parameters from GAMS database
interested_symbols = ['K_index', 'L_index']  # Add more symbols as needed
interested_years = [str(t)]  # Add more years as needed
interested_regions = ['Netherlands']
# Assuming 'db' is your GAMS database object
filtered_data = []

# Iterate through all symbols in the database
for symbol in db:
    if symbol.name in interested_symbols:
        print(f"Symbol: {symbol.name}")
        for rec in symbol:
            if len(rec.keys) == 4:  # Assuming the order is region, sector, education, year
                region, sector, edu, year = rec.keys
            else:  # If 'education' is not present, assign NaN to edu
                region, sector, year = rec.keys
                edu = np.nan  # Use numpy's NaN for missing values
            
            # Now include the 'education' dimension in your filtering and data appending
            if region in interested_regions and year in interested_years:
                filtered_data.append({
                    'Region': region,
                    'Year': year,
                    'Sector': sector,
                    'Education': edu,  # Include 'education' regardless of its presence
                    'Value': rec.value
                })

# Convert the filtered data to a pandas DataFrame
df = pd.DataFrame(filtered_data)